GitHub<br>
https://github.com/open-mmlab/mmgeneration<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/MMGeneration_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## Githubからソースコード取得

In [ ]:
%cd /content

!git clone https://github.com/open-mmlab/mmgeneration.git

## ライブラリのインストール

In [ ]:
%cd /content/mmgeneration

!pip install moviepy
!pip3 install openmim
!mim install mmcv-full

!pip install -e .[all]

# synthesized images with unconditional GAN

In [ ]:
import mmcv
from mmgen.apis import init_model, sample_unconditional_model

# Specify the path to model config and checkpoint file
config_file = 'configs/styleganv2/stylegan2_c2_ffhq_1024_b4x8.py'
# you can download this checkpoint in advance and use a local file path.
checkpoint_file = 'https://download.openmmlab.com/mmgen/stylegan2/stylegan2_c2_ffhq_1024_b4x8_20210407_150045-618c9024.pth'

device = 'cuda:0'
# init a generatvie
model = init_model(config_file, checkpoint_file, device=device)
# sample images
fake_imgs = sample_unconditional_model(model, 4)

In [ ]:
from torchvision import utils
import matplotlib.pyplot as plt
import os
from IPython.display import Image, display

results = (fake_imgs[:, [2, 1, 0]] + 1.) / 2.
save_path = './outputs/unconditional/results.jpg'

# save images
mmcv.mkdir_or_exist(os.path.dirname(save_path))
utils.save_image(
    results, save_path, ncol=1, padding=0)

# display
display(Image(save_path))

# synthesized images with conditional GAN

In [ ]:
from mmgen.apis import init_model, sample_conditional_model

# Specify the path to model config and checkpoint file
config_file = 'configs/sagan/sagan_128_woReLUinplace_noaug_bigGAN_Glr-1e-4_Dlr-4e-4_ndisc1_imagenet1k_b32x8.py'
# you can download this checkpoint in advance and use a local file path.
checkpoint_file = 'https://download.openmmlab.com/mmgen/sagan/sagan_128_woReLUinplace_noaug_bigGAN_imagenet1k_b32x8_Glr1e-4_Dlr-4e-4_ndisc1_20210818_210232-3f5686af.pth'

device = 'cuda:0'
# init a generatvie
model = init_model(config_file, checkpoint_file, device=device)

# sample images with specific labels
# label https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
fake_imgs = sample_conditional_model(model, 4, label=[0, 1, 2, 3])

In [ ]:
results = (fake_imgs[:, [2, 1, 0]] + 1.) / 2.
save_path = './outputs/conditional/results.jpg'

# save images
mmcv.mkdir_or_exist(os.path.dirname(save_path))
utils.save_image(
    results, save_path, ncol=1, padding=0)

# display
display(Image(save_path))

# image translation

In [ ]:
from mmgen.apis import init_model, sample_img2img_model

config_file = 'configs/pix2pix/pix2pix_vanilla_unet_bn_wo_jitter_flip_edges2shoes_b1x4_190k.py'
checkpoint_file = 'https://download.openmmlab.com/mmgen/pix2pix/refactor/pix2pix_vanilla_unet_bn_wo_jitter_flip_1x4_186840_edges2shoes_convert-bgr_20210902_170902-0c828552.pth'

# Specify the path to image you want to translate
image_path = 'tests/data/paired/test/33_AB.jpg'
device = 'cuda:0'
# init a generatvie
model = init_model(config_file, checkpoint_file, device=device)
# translate a single image
translated_image = sample_img2img_model(model, image_path, target_domain='photo')

In [ ]:
results = (translated_image[:, [2, 1, 0]] + 1.) / 2.
save_path = './outputs/translation/results.jpg'

# save images
mmcv.mkdir_or_exist(os.path.dirname(save_path))
utils.save_image(
    results, save_path, ncol=1, padding=0)

# display
display(Image(save_path))

# Interpolation

In [ ]:
!python apps/interpolate_sample.py \
  configs/styleganv2/stylegan2_c2_ffhq_256_b4x8_800k.py \
  https://download.openmmlab.com/mmgen/stylegan2/stylegan2_c2_ffhq_256_b4x8_20210407_160709-7890ae1f.pth \
  --export-video \
  --samples-path outputs/interpolation \
  --endpoint 6 \
  --interval 60 \
  --space z \
  --seed 12 \
  --sample-cfg truncation=0.8

In [ ]:
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

# display
clip = VideoFileClip('/content/mmgeneration/outputs/interpolation/lerp.mp4')
clip = resize(clip, height=420)
clip.ipython_display()